Template notebook for one type of model.

In [209]:
import pandas as pd
from xgboost import XGBClassifier,XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBRegressor, XGBClassifier
from statistics import mean
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
# Other imports

In [186]:
# Load data
data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")
data_test = pd.read_csv("test_data_enriched.csv",index_col="Claim Identifier")

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_5244/1043285265.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")


In [187]:
le = LabelEncoder()

In [188]:
#Label inconding our target variable 
data["Claim Injury Type"] = le.fit_transform(data["Claim Injury Type"])

In [189]:
def impute_missing_values(data, target_column, algorithm):
    
    # Separating the missing values from the non missing values
    available_data = data[data[target_column].notna()]
    missing_data = data[data[target_column].isna()]

    # Making sure there is enough data to input 
    if len(available_data) == 0 or len(missing_data) == 0:
        return data

    # Separating the target column from the rest 
    X_available = available_data.drop(columns=[target_column])
    y_available = available_data[target_column]

    # Training the model with the available data
    model = algorithm
    model.fit(X_available, y_available)

    # Prediting the missing values
    X_missing = missing_data.drop(columns=[target_column])
    predicted_values = model.predict(X_missing)

    # Inputing the missing values with the predictions
    data.loc[data[target_column].isna(), target_column] = predicted_values

    return data

In [190]:
def handle_outliers(data, target_column):
    lower_quantile = 0.25
    upper_quantile = 0.75
    multiplier = 1.5
    # Calculate Q1, Q3, and IQR
    Q1 = data[target_column].quantile(lower_quantile)
    Q3 = data[target_column].quantile(upper_quantile)
    IQR = Q3 - Q1
    
    # Define outlier bounds
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    # Replace outliers with NaN
    data[target_column] = data[target_column].apply(
        lambda x: np.nan if x < lower_bound or x > upper_bound else x
    )
    
    return data

In [191]:
# Scaling function
def scale_numerical(column, X_train, X_val, scaler):
    # Ajusta a escala da coluna numérica convertendo-a para 2D
    X_train[column] = pd.DataFrame(scaler.fit_transform(X_train[[column]]))
    X_val[column] = pd.DataFrame(scaler.transform(X_val[[column]]))


In [192]:
def categorical_ordinal_encode(X_train, X_val,column):
    # Define a function to categorize each carrier based on its claim count
    count = X_train['Carrier Name'].value_counts()
    def categorize_claims(count):
        if count >= 40000:
            return 2
        elif 4000 <= count < 40000:
            return 1
        else:
            return 0

    # Apply the categorization to create a mapping dictionary
    carrier_category_map = count.apply(categorize_claims)

    # Map the `Carrier Name` to the new `Carrier Claim Category`
    X_train['Carrier Claim Category'] = X_train['Carrier Name'].map(carrier_category_map)


In [193]:
# Categorical encoder function
def categorical_prop_encode(X_train, X_val, feature):
    proportion = X_train[feature].value_counts(normalize = True)  # Get the porportion of each category
    X_train[feature] = X_train[feature].map(proportion)  # Map the porportions in the column
    X_val[feature] = X_val[feature].map(proportion) # Do the same for the valid dataset
    X_val[feature] = X_val[feature].fillna(0)  # Handle categories in X_val not seen in X_train with 0


In [ ]:
def cv_scores(model, X, y, num_features, cat_features, num_imputing_algorithm= XGBRegressor() , cat_imputing_algorithm = XGBClassifier(), scalling_outlier = True, scaler = MinMaxScaler()):
    ''' Takes as argument the model used, the predictors and the target, the models used for imputing numerical and categorical 
      features, if any scaling and outlier removed should be performed, and what scaling method should be used.
     Then it returns the results obtained from the stratified cross validation for the given model'''
    
    skf = StratifiedKFold(n_splits=5)
    
    # Generating the lists to store our results
    precision_scores_train = []
    precision_scores_val = []   
    recall_scores_train = []  
    recall_scores_val = []
    f1_scores_train = []    
    f1_scores_val = []
    index = [f'Fold {i}' for i in range(1,6)]
    index.append("Average")
    
    for train_index, test_index in skf.split(X, y):
        # Dividing our data in validation and train
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

 
        #Filling num missing values
        for column in num_features:
            impute_missing_values(X_train, column, num_imputing_algorithm)
            impute_missing_values(X_val, column, num_imputing_algorithm)

        #Filling cat missing values
        impute_missing_values(X_train, "Alternative Dispute Resolution", cat_imputing_algorithm)
        impute_missing_values(X_val, "Alternative Dispute Resolution", cat_imputing_algorithm)


        # Removing inconsistencies on the train
        inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
        X_train = X_train.loc[~X_train.index.isin(inconsistent)]
        y_train = y_train.loc[~y_train.index.isin(inconsistent)]

        #Performing scaling and outlier treatment dependent on the boolean
        if scalling_outlier:
            for column in num_features:
                handle_outliers(X_train, column)
                scale_numerical(column,X_train, X_val, scaler)

        """# Creating an ordinal variable
        categorical_ordinal_encode(X_train, X_val, num_features)"""


        # Categorical Prop Encoding
        for cat_feature in cat_features:
            categorical_prop_encode(X_train, X_val, cat_feature)
            
        
        # Training the classification model
        model.fit(X_train, y_train)

        
        # Making the predictions for the training and validation data
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
    
        
        # Calculating and storing the scores
        precision_scores_train.append(precision_score(y_train, pred_train, average='macro'))
        precision_scores_val.append(precision_score(y_val, pred_val, average='macro'))
        recall_scores_train.append(recall_score(y_train, pred_train, average='macro'))
        recall_scores_val.append(recall_score(y_val, pred_val, average='macro'))
        f1_scores_train.append(f1_score(y_train, pred_train, average='macro'))
        f1_scores_val.append(f1_score(y_val, pred_val, average='macro'))

    
    precision_scores_train.append(mean(precision_scores_train))
    precision_scores_val.append(mean(precision_scores_val))
    recall_scores_train.append(mean(recall_scores_train))
    recall_scores_val.append(mean(recall_scores_val))
    f1_scores_train.append(mean(f1_scores_train))
    f1_scores_val.append(mean(f1_scores_val))


    # Storing the results in a dataframe
    model_results = pd.DataFrame(data={
        'Train_precision': precision_scores_train,
        'Test_precision': precision_scores_val,
        'Train_recall': recall_scores_train,
        'Test_recall': recall_scores_val,
        'Train_f1_score': f1_scores_train,
        'Test_f1_score': f1_scores_val,
    }, index=index)
    
    return model_results



In [ ]:
"""def test_prediction(model, X, y , test, num_inputing_algorithm= XGBRegressor() , cat_inputing_algorithm = XGBClassifier(),scalling_outlier = True, scaler = MinMaxScaler()):

    X_train, X_val,y_train, y_val = train_test_split(X,y,
                                                train_size = 0.8, 
                                                shuffle = True, 
                                                stratify = y)
    #Performing scaling and outlier treatment dependent on the boolean
    if scalling_outlier:
        for column in num_features:
            handle_outliers(X_train, column)
            scale_numerical(column,X_train, X_val, scaler)

    # Missing value inputation
    #Filling num missing values
    for column in num_features:
        impute_missing_values(X_train, column, num_inputing_algorithm)
        impute_missing_values(X_val, column, num_inputing_algorithm)
        impute_missing_values(test, column, num_inputing_algorithm)

    #Filling cat missing values
    impute_missing_values(X_train, "Alternative Dispute Resolution", cat_inputing_algorithm)
    impute_missing_values(X_val, "Alternative Dispute Resolution", cat_inputing_algorithm)
    impute_missing_values(test, "Alternative Dispute Resolution", cat_inputing_algorithm)

    # Removing inconsistencies on the train
    inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
    X_train.drop(inconsistent, inplace=True)
    y_train.drop(inconsistent, inplace=True)

    # Creating an ordinal variable
    for num_feature in num_features:
        categorical_ordinal_encode(X_train, X_val,num_feature)
        categorical_ordinal_encode(X_train, test,num_feature)

    # Categorical Prop Encoding
    for cat_feature in cat_features:
        categorical_prop_encode(X_train, X_val, cat_feature)
        categorical_prop_encode(X_train, test, cat_feature)
    
    # Fitting the model
    model.fit(X_train, y_train)

    # Veryfing if the model is performing as expected
    pred_val = model.predict(X_val)
    print(f1_score(y_val, pred_val, average='macro'))

    # Using the model to make prediction on the test dataset
    pred_test = model.predict(test)

    # Inversing the encoding of our target variable 
    pred_test = le.inverse_transform(pred_test)

    # Making a dataframe with the indexes of data_test and predictions converted back to strings
    submission_df = pd.DataFrame({
        "Claim Injury Type": pred_test
    }, index=data_test.index)
    
    return submission_df


    """

Temos de rever as num features e cat features

In [195]:
num_features = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents',
                "Accident Year","Accident Month","Accident Day","Accident DayOfWeek","Assembly Date DSA",
                "C-2 Date DSA","C-3 Date DSA","First Hearing Date DSA" ,"Accident Year",
    "Accident Month",
    "Accident Day",
    "Accident DayOfWeek"]

In [196]:
cat_features = [
    "Alternative Dispute Resolution",
    "Carrier Name",
    "Carrier Type",
    "County of Injury",
    "District Name",
    "Gender",
    "Industry Code",
    "Medical Fee Region",
    "WCIO Cause of Injury Code",
    "WCIO Nature of Injury Code",
    "WCIO Part Of Body Code",
    "Age at Injury Category",
    "Carrier Claim Category",
    "Body Section"
]

temos de dropar as colunas e por no pre-processing

In [197]:
# Dropping redundant variables that carry almost the same information (are extremely correlated (|0.8|))
# We believe it was better to keep Age at Injury than birth year since it should be more related to the injury claim type (it will be tested later)
# The same logic was applied to dropping the other two dates and two DSA variables since we believe Accident date to be more important
# Para `data`
data = data.loc[:, ~data.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]

# Para `data_test`
data_test = data_test.loc[:, ~data_test.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]


for col in ['Birth Year',"Assembly Date DSA", "First Hearing Date DSA"]:
    num_features.remove(col)

In [198]:
#Since the codes always seem to provide the same or more information than the descriptions (have more categories),
#and the codes are consistent (always only having 1 description for code, while descriptions may have multiple codes)
#we will drop the description columns.
data.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)
data_test.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)


In [199]:
# Removing Zip Code for reason meantion in pre-processement
data.drop(['Zip Code'], axis=1, inplace = True)
data_test.drop(['Zip Code'], axis=1 , inplace = True)

In [200]:
X = data.drop(["Claim Injury Type"], axis = 1)

In [201]:
y = data["Claim Injury Type"]

In [ ]:
# ------------------------

In [36]:
selected_features = ['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution', 'Attorney/Representative', 'Average Weekly Wage', 'C-3 Date', 'Carrier Name', 'Carrier Type', 'County of Injury',
                      'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender', 'IME-4 Count', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 
                      'WCIO Part Of Body Code', 'Number of Dependents', 'Accident Year', 'Accident Month', 'Accident Day', 'Accident DayOfWeek', 'C-2 Date DSA', 'C-3 Date DSA',
                         'Age at Injury Category', # 'Accident Date_missing', 'First Hearing Date_missing', 'C-3 Date_missing', 'Assembly Date_missing', 'C-2 Date_missing',
                        'Carrier Claim Category', 'Body Section']

# Select Categorical features
selected_cat_features = ['Alternative Dispute Resolution', 'Attorney/Representative', 'Carrier Name', 'Carrier Type', 'County of Injury',
                        'COVID-19 Indicator', 'District Name', 'Gender', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code',
                        'WCIO Part Of Body Code', 'Carrier Claim Category', 'Body Section'] # based on RFE

# Select numerical features
selected_num_features = ['Age at Injury', 'Average Weekly Wage','IME-4 Count','Number of Dependents',
                         'Accident Year','Accident Month', 'Accident Day', 'Accident DayOfWeek',
                         'C-2 Date DSA', 'C-3 Date DSA'] # based on RFE

In [211]:
xgb_results = cv_scores(XGBClassifier(), X, y,num_features,cat_features,scalling_outlier =False)
# look at results

/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_5244/1444211760.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = X_val[feature].map(proportion) # Do the same for the valid dataset
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_5244/1444211760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = X_val[feature].fillna(0)  # Handle categories in X_val not seen in X_train with 0
/var/folders/fb/mypr4jd11nj8y_wzms74rj2w0000gn/T/ipykernel_5244/1444211760.

In [208]:
xgb_results

,Train_precision,Test_precision,Train_recall,Test_recall,Train_f1_score,Test_f1_score
Fold 1,0.797052,0.369771,0.603691,0.425841,0.640727,0.295501
Fold 2,0.806168,0.267869,0.625445,0.254185,0.663898,0.246825
Fold 3,0.800956,0.243939,0.625061,0.214755,0.664659,0.219250
Fold 4,0.804798,0.253076,0.607866,0.207213,0.644495,0.217018
Fold 5,0.800574,0.467524,0.585593,0.194822,0.618541,0.144610
Average,0.801910,0.320436,0.609531,0.259363,0.646464,0.224641


In [ ]:
# Test prediction
    # Choose best model from KFold above
    # Train model on whole train and predict test data
    submission = test_prediction(XGBClassifier(),X[selected_features],y,data_test)
    submission